In [55]:
# Sudoku Puzzle Solver -- full CNN model
# (c) AlexPfaff June 2025

import tensorflow as tf
from tensorflow import keras
from keras import Model, Input
from keras.layers import Embedding, Conv2D, Activation, LSTM, Bidirectional, Dense, Reshape, Flatten, Lambda
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from PsQ_Grid import Grid
from PsQ_GridCollection import GridCollection as GC
import numpy as np

from typing import List, Tuple, Literal, Optional
from sklearn.model_selection import train_test_split
from tqdm import tqdm 


In [56]:
def make_fullCNNModel():
    vocab_size = 9  # digits 1-9
    embedding_dim = 16

    inputs = Input(shape=(9, 9), dtype='int32')  # (batch, 9, 9)

    # Learnable embedding for each digit 0-9 (0 = masked)
    x = Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, mask_zero=False)(inputs)  # → (batch, 9, 9, embed_dim)

    # Convolutional feature extraction -- standard
    # x = Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    # x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    # x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)


    # Convolutional feature extraction -- power3
    x = Conv2D(81, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(81, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(81, (3, 3), padding='same', activation='relu')(x)
    x = Conv2D(81, (3, 3), padding='same', activation='relu')(x)


    # Output: predict 1 of 9 digits at each cell (softmax over depth)
    x = Conv2D(vocab_size, (1, 1), padding='same')(x)
    outputs = Activation('softmax')(x)  # shape: (batch, 9, 9, 9)

    model = Model(inputs, outputs)
    model.summary()

    optimizer = Adam(
        learning_rate=0.0012,       
        beta_1=0.9,                
        beta_2=0.999,              
        epsilon=1e-7              
    )

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [57]:
class Grid_NN_mulitple_k_blanks_Solver:
    """ Trains Sudoku solver (= full CNN model);
        input data: n x 9 x 9 arrays (= Sudoku grids with variable number of blanks = 0),
        raw output = model predictions: n x 9 x 9 x 9 (= n x 9 x 9 softmaxed probability distributions over digits 1-9)  
    """

    def __init__(self, 
                 gridCollection: np.ndarray, 
                 max_k_blanks = 60,                     # max number of blanks
                 include_parentSet: bool = True         # do we include the naked parent dataset in the training data?
                 ) -> None:           
        self.gridCollection = gridCollection.copy().reshape(-1, 81)
        self.size = self.gridCollection.shape[0]
        self.labels = gridCollection.copy().reshape(-1, 81) if include_parentSet else np.empty((0, 81), dtype=self.gridCollection.dtype) 
        self.maskedGridCollection = gridCollection.copy().reshape(-1, 81) if include_parentSet else np.empty((0, 81), dtype=self.gridCollection.dtype) 
        self.max_k_blanks = max_k_blanks
        self.GRIDSIZE = 81

        assert self.maskedGridCollection.shape[0] == self.labels.shape[0]

        self.model = make_fullCNNModel()

        self._mask_normal()
        self._mask_largeGaps()
        self._prep_split_Data()

    @property
    def datasize(self):
        return self.maskedGridCollection.shape[0]

    def _mask_normal(self):
        """ Iterates several times through the dataset creating grids with 1--{max_k_blanks} blanks = 0s """
        n_samples = self.size * 4
        output = np.empty((n_samples, self.GRIDSIZE), dtype=self.gridCollection.dtype)
        labels = np.empty((n_samples, self.GRIDSIZE), dtype=self.gridCollection.dtype)
        
        for iteration in tqdm(range(n_samples)): 
            idx = iteration % self.size 
            k = iteration % self.max_k_blanks + 1 
            mask = np.random.choice(range(self.GRIDSIZE), k, replace=False)
            grid = self.gridCollection[idx].copy()
            grid[mask] = 0 
            output[iteration] = grid
            labels[iteration] = self.gridCollection[idx].copy()

        self.maskedGridCollection = np.concatenate([self.maskedGridCollection, output])
        self.labels = np.concatenate([self.labels, labels])


    def _mask_largeGaps(self):
        """ Iterates several times through the dataset creating grids with 2/3 * {max_k_blanks} -- {max_k_blanks} blanks = 0s """
        n_samples = self.size * 4
        output = np.empty((n_samples, self.GRIDSIZE), dtype=self.gridCollection.dtype)
        labels = np.empty((n_samples, self.GRIDSIZE), dtype=self.gridCollection.dtype)

        for iteration in tqdm(range(n_samples)): 
            idx = iteration % self.size 
            low = int(self.max_k_blanks *(3/5) + 2)
            diff = self.max_k_blanks - low + 1
            k = iteration % low + diff   

            mask = np.random.choice(range(self.GRIDSIZE), k, replace=False)
            grid = self.gridCollection[idx].copy()
            grid[mask] = 0 
            output[iteration] = grid
            labels[iteration] = self.gridCollection[idx].copy()

        self.maskedGridCollection = np.concatenate([self.maskedGridCollection, output])
        self.labels = np.concatenate([self.labels, labels])

    def _prep_split_Data(self):
        self.maskedGridCollection = self.maskedGridCollection.reshape(-1, 9, 9)
        self.labels = self.labels.reshape(-1, 9, 9)
        self.labels -= 1

        X_train, X_test, y_train, y_test = train_test_split(
            self.maskedGridCollection, self.labels)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train)

        print(X_train.shape, X_test.shape, X_val.shape)

        self.X_train, self.X_val, self.X_test = X_train, X_val, X_test
        self.y_train, self.y_val, self.y_test = y_train, y_val, y_test
#        self.X_train, self.X_val, self.X_test = X_train.reshape(-1, 9, 9), X_val.reshape(-1, 9, 9), X_test.reshape(-1, 9, 9)
#        self.y_train, self.y_val, self.y_test = y_train.reshape(-1, 9, 9), y_val.reshape(-1, 9, 9), y_test.reshape(-1, 9, 9)



    def get_data(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """Returns the full dataset splits: (X_train, y_train, X_val, y_val, X_test, y_test)"""
        return self.X_train, self.y_train, self.X_val, self.y_val, self.X_test, self.y_test


    def fit(self):
        early_stop = EarlyStopping(
            monitor='val_loss',              
            patience=3,  
            restore_best_weights=True,
            verbose=2
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.51,          
            patience=2,        
            min_lr=1e-5,
            verbose=2
        )

        X_train, y_train, X_val, y_val, _, _ = self.get_data()

        print(X_train.shape, X_train.min(), X_train.max())
        print(y_train.shape, y_train.min(), y_train.max())
        print(X_val.shape, X_val.min(), X_val.max())
        print(y_val.shape, y_val.min(), y_val.max())


        self.history = self.model.fit(X_train, y_train,
                                    batch_size=256, #  128
                                    epochs=15,
                                    validation_data=(X_val, y_val),
                                    callbacks=[early_stop, reduce_lr],
                                    shuffle=True)
                                


    def evaluate(self): 
        self.model.evaluate(self.X_test, self.y_test)



    def gimme_k_blanks(self, k):
        X = self.maskedGridCollection.copy()
        zero_counts = (X == 0).sum(axis=1)
        return np.where(zero_counts == k)[0]








    def play_the_game(self, grid: np.array):

        while grid.min() == 0:
            y_preds = self.model.predict(grid[None, ...])
            pred = y_preds[0]                      
            i, j, val = self._find_most_confident_prediction(pred, grid)

            grid[i, j] = val

            self.play_the_game(grid)

        return grid


    def _find_most_confident_prediction(self, x_pred, x_input):
        """
        Find the (i, j, d) with the highest confidence in x_pred,
        considering only those (i, j) where x_input[i, j] == 0.
        
        Parameters:
            x_pred : np.ndarray of shape (9, 9, 9)
                Model output probabilities.
            x_input : np.ndarray of shape (9, 9)
                Input Sudoku grid with 0s in cells to be filled.

        Returns:
            i, j, d : int
                Position (i, j) and digit d (1–9) of the most confident eligible prediction.
        """
        # Create mask for positions where x_input == 0, then broadcast to shape (9, 9, 9)
        mask = (x_input == 0)[:, :, None]                     # shape (9, 9, 1)
        mask = np.broadcast_to(mask, x_pred.shape)           # shape (9, 9, 9)

        masked_probs = np.where(mask, x_pred, -np.inf)       # ignore positions with filled digits
        flat_index = np.argmax(masked_probs)
        i, j, d = np.unravel_index(flat_index, (9, 9, 9))
        return i, j, d + 1   # convert class index (0–8) to digit (1–9)

#    def _find_most_confident_prediction(self, x):
#        """
#        Given a single model output of shape (9, 9, 9), find the (i, j) cell and predicted digit
#        with the highest confidence score across the entire grid.
#        """
#        flat_index = np.argmax(x)                           # index in flattened (729,) array
#        i, j, d = np.unravel_index(flat_index, (9, 9, 9))   # convert to 3D index
#        return i, j, d + 1                                  # shift class index to digit
        




In [58]:
rnd_idx = np.random.randint(6**8)
gc = GC.from_scratch()
gc.activate_HorizontalSeries(50)  
gc = gc.activeGrid.copy()


initialize @Level 1:  [7, 9, 1, 6, 2, 5, 8, 3, 4]
success    @Level 2:  (5, 2, 6, 8, 3, 4, 7, 9, 1)
success    @Level 3:  (4, 8, 3, 1, 7, 9, 5, 6, 2)
success    @Level 4:  (9, 7, 2, 3, 8, 6, 4, 1, 5)
success    @Level 5:  (8, 5, 4, 2, 9, 1, 3, 7, 6)
success    @Level 5:  (8, 5, 4, 2, 9, 1, 6, 7, 3)
success    @Level 5:  (8, 5, 4, 9, 1, 2, 3, 7, 6)
success    @Level 5:  (8, 5, 4, 9, 1, 2, 6, 7, 3)
success    @Level 5:  (8, 5, 4, 9, 1, 7, 3, 2, 6)
success    @Level 5:  (8, 5, 4, 9, 1, 7, 6, 2, 3)
success    @Level 5:  (8, 5, 4, 7, 9, 1, 3, 2, 6)
success    @Level 5:  (8, 5, 4, 7, 9, 1, 6, 2, 3)
success    @Level 5:  (8, 4, 5, 2, 9, 1, 3, 7, 6)
success    @Level 5:  (8, 4, 5, 2, 9, 1, 6, 7, 3)
success    @Level 5:  (8, 4, 5, 9, 1, 2, 3, 7, 6)
success    @Level 5:  (8, 4, 5, 9, 1, 2, 6, 7, 3)
success    @Level 5:  (8, 4, 5, 9, 1, 7, 3, 2, 6)
success    @Level 5:  (8, 4, 5, 9, 1, 7, 6, 2, 3)
success    @Level 5:  (8, 4, 5, 7, 9, 1, 3, 2, 6)
success    @Level 5:  (8, 4, 5, 7, 9, 1, 6, 2, 3)


100%|██████████| 1296/1296 [00:04<00:00, 274.62it/s]


In [ ]:
multiple_k_solver = Grid_NN_mulitple_k_blanks_Solver(gc)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 9, 9)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 9, 9, 16)       │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 9, 9, 81)       │        11,745 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 9, 9, 81)       │        59,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 81)       │        59,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 9, 9, 81)       │        59,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 9, 9, 9)        │           738 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 9, 9, 9)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 190,033 (742.32 KB)

 Trainable params: 190,033 (742.32 KB)

 Non-trainable params: 0 (0.00 B)

100%|██████████| 1451520/1451520 [00:18<00:00, 80210.32it/s]


(1837080, 9, 9) (816480, 9, 9) (612360, 9, 9)


In [ ]:
print(f"size dataset: {multiple_k_solver.datasize}\n")
multiple_k_solver.fit()

size dataset: 3265920

(1837080, 9, 9) 0 9
(1837080, 9, 9) 0 8
(612360, 9, 9) 0 9
(612360, 9, 9) 0 8
Epoch 1/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 645s 90ms/step - accuracy: 0.8055 - loss: 0.5087 - val_accuracy: 0.8698 - val_loss: 0.3337 - learning_rate: 0.0012
Epoch 2/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 631s 88ms/step - accuracy: 0.8871 - loss: 0.2874 - val_accuracy: 0.9037 - val_loss: 0.2502 - learning_rate: 0.0012
Epoch 3/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 628s 87ms/step - accuracy: 0.9140 - loss: 0.2236 - val_accuracy: 0.9324 - val_loss: 0.1775 - learning_rate: 0.0012
Epoch 4/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 630s 88ms/step - accuracy: 0.9324 - loss: 0.1793 - val_accuracy: 0.9369 - val_loss: 0.1690 - learning_rate: 0.0012
Epoch 5/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 634s 88ms/step - accuracy: 0.9428 - loss: 0.1525 - val_accuracy: 0.9456 - val_loss: 0.1456 - learning_rate: 0.0012
Epoch 6/15
7177/7177 ━━━━━━━━━━━━━━━━━━━━ 629s 88ms/step - accuracy: 0.9494 - loss: 0.1350 - val_accuracy: 0.9521 

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = multiple_k_solver.get_data()

In [ ]:
# evaluating training data / validation data / test data
multiple_k_solver.model.evaluate(X_train, y_train)
multiple_k_solver.model.evaluate(X_val, y_val)
multiple_k_solver.model.evaluate(X_test, y_test)

57409/57409 ━━━━━━━━━━━━━━━━━━━━ 362s 6ms/step - accuracy: 0.9665 - loss: 0.0869
19137/19137 ━━━━━━━━━━━━━━━━━━━━ 112s 6ms/step - accuracy: 0.9665 - loss: 0.0870
25515/25515 ━━━━━━━━━━━━━━━━━━━━ 156s 6ms/step - accuracy: 0.9664 - loss: 0.0872


[0.0872543677687645, 0.9664185643196106]

# Save model and data (= blanked grids) to file


In [ ]:
multiple_k_solver.model.save("multiple_k_solver_pow3.keras") 

dat = multiple_k_solver.maskedGridCollection
#lab = multiple_k_solver.labels
np.save("sudoku_data_pow3.npy", dat)
#data = np.load("sudoku_data.npy")
#np.save("sudoku_labels.npy", lab)
#labels = np.load("sudoku_labels.npy")


# NB: you can call the model / data whatever you want, 
# but if you want to use them in the GUI, be sure to update the filenames in PsQ_GUI in lines 49/50 accordingly!

In [ ]:
y_preds_train = multiple_k_solver.model.predict(X_train)
y_preds_val = multiple_k_solver.model.predict(X_val)
y_preds_test = multiple_k_solver.model.predict(X_test)

57409/57409 ━━━━━━━━━━━━━━━━━━━━ 241s 4ms/step
19137/19137 ━━━━━━━━━━━━━━━━━━━━ 88s 5ms/step
25515/25515 ━━━━━━━━━━━━━━━━━━━━ 116s 5ms/step


In [ ]:
train_p = np.argmax(y_preds_train, axis=-1)
val_p = np.argmax(y_preds_val, axis=-1)
test_p = np.argmax(y_preds_test, axis=-1)

In [ ]:
# checking a full prediction

g_idx = np.random.randint(X_test.shape[0])

print(X_test[g_idx])
print( )
print(y_test[g_idx]+1)
print()
print(test_p[g_idx]+1)
print()
print()
print("Is the predicted (full) grid correct? Sum should be 81 x True")
print(np.all(y_test[g_idx].reshape(-1, 81) == test_p[g_idx].reshape(-1, 81), axis=1))
print((y_test[g_idx] == test_p[g_idx]).sum())

[[5 3 1 2 6 0 7 0 4]
 [2 0 0 9 0 7 0 1 5]
 [7 9 4 0 1 5 2 0 0]
 [6 2 3 0 8 0 1 7 0]
 [9 1 5 7 2 0 8 4 6]
 [8 0 7 0 9 1 5 2 3]
 [0 0 2 8 3 6 0 5 0]
 [3 5 6 1 0 0 0 8 2]
 [1 8 0 4 5 0 6 0 7]]

[[5 3 1 2 6 8 7 9 4]
 [2 6 8 9 4 7 3 1 5]
 [7 9 4 3 1 5 2 6 8]
 [6 2 3 5 8 4 1 7 9]
 [9 1 5 7 2 3 8 4 6]
 [8 4 7 6 9 1 5 2 3]
 [4 7 2 8 3 6 9 5 1]
 [3 5 6 1 7 9 4 8 2]
 [1 8 9 4 5 2 6 3 7]]

[[5 3 1 2 6 8 7 9 4]
 [2 6 8 9 4 7 3 1 5]
 [7 9 4 3 1 5 2 6 8]
 [6 2 3 5 8 4 1 7 9]
 [9 1 5 7 2 3 8 4 6]
 [8 4 7 6 9 1 5 2 3]
 [4 7 2 8 3 6 9 5 1]
 [3 5 6 1 7 9 4 8 2]
 [1 8 9 4 5 2 6 3 7]]


Is the predicted (full) grid correct? Sum should be 81 x True
[ True]
81


In [ ]:
# sanity check -- is the accuracy score really justified, the grids correctly predicted?
matches               = np.all(test_p.reshape(-1, 81) == y_test.reshape(-1, 81), axis=1)    
fully_identical_grids = np.sum(matches)     
matching_indices      = np.where(matches)[0]  

print(f"Number of total grid matches -- prediction == label:\n {fully_identical_grids} = {round(fully_identical_grids/y_test.shape[0] * 100, 5)}%")


Number of total grid matches -- prediction == label:
 429306 = 52.5801%


## Accuracy score of around 50% (for 81 out of 81) does not exactly inspire trust .... 
However, the model won't be used at face value, only one out of 81 predictions will be used:  
model.predict() output shape (n, 9, 9, 9) = n x 9 x 9 probability distributions, 81 per grid, but we are not interested in 81 out of 81!
* the ones that have a non-zero value in the input will be ignored -- those values we already have, they need not be predicted
* amongst the outputs that predict a value for a blank, only the one with the highest overall probability == best argmax is chosen 
* e.g. if argmax((i, j)) > argmax((x, y)) then the value predicted for position (i, j) will be the unique prediction

In other words, the modified predict() method, picks only one value and discards 80. 

# The Most Crucial Part: best argmax wins! <br>

## Recursive Procedure 

*  probs = model.predict(G_with_k_blanks)         --> 9 x 9 softmaxed probability distributions --> 81 argmaxes
*  probs[i, j] only considered iff G[i, j] == 0
*  x = argmax(probs) -- best argmax of 81
*  (i,j) = coordinates(x)
*  v = val(x) 
* G[i, j] = v   (<--> k -= 1)
* probs = model.predict(G_with_k-1_blanks) 
* Repeat until k == 0

In [ ]:
def pick_k_blanks_grid(puzzles: np.array, k = 30): 
    zero_counts = (puzzles == 0).sum(axis=(1, 2))   
    mask =  zero_counts == k 
    candidates = puzzles[mask].copy()
    idx = np.random.randint(candidates.shape[0])
    return candidates[idx] 


def _find_most_confident_prediction(x_pred, x_input):
    """
    Find the (i, j, d) with the highest confidence in x_pred,
    considering only those (i, j) where x_input[i, j] == 0.
    
    Parameters:
        x_pred : np.ndarray of shape (9, 9, 9)
            Model output probabilities.
        x_input : np.ndarray of shape (9, 9)
            Input Sudoku grid with 0s in cells to be filled.

    Returns:
        i, j, d : int
            Position (i, j) and digit d (1–9) of the most confident eligible prediction.
    """
    # Create mask for positions where x_input == 0, then broadcast to shape (9, 9, 9)
    mask = (x_input == 0)[:, :, None]                     # shape (9, 9, 1)
    mask = np.broadcast_to(mask, x_pred.shape)           # shape (9, 9, 9)

    masked_probs = np.where(mask, x_pred, -np.inf)       # ignore positions with filled digits
    flat_index = np.argmax(masked_probs)
    i, j, d = np.unravel_index(flat_index, (9, 9, 9))
    return i, j, d + 1   # convert class index (0–8) to digit (1–9)


def play_the_game(grid: np.array):

    if grid.min() > 0:
        return None, None, None, grid

    y_preds = multiple_k_solver.model.predict(grid[None, ...])
    pred = y_preds[0]                      
    i, j, val = _find_most_confident_prediction(pred, grid)

    grid[i, j] = val
    return  (i, j), val, grid

g = Grid()
myGrid = pick_k_blanks_grid(X_test)

In [ ]:
# if you run this cell for the first time, you will see two grids below (before / after insertion)
# to "play" the game, run this cell until there is only one grid left (= no more zeros!), or gridCheck() returns False 
if (myGrid == 0).sum() > 0:
    print(myGrid)
    c, v, myGrid = play_the_game(myGrid)
g.insert(myGrid.flatten())
g.showGrid()
print(g.gridCheckZero())


-------------------------
| 2 7 5 | 6 8 4 | 1 9 3 | 
| 6 8 4 | 9 3 1 | 7 5 2 | 
| 1 9 3 | 7 5 2 | 6 8 4 | 
-------------------------
| 8 6 7 | 2 4 3 | 5 1 9 | 
| 9 5 2 | 1 6 7 | 4 3 8 | 
| 4 3 1 | 8 9 5 | 2 6 7 | 
-------------------------
| 3 1 6 | 4 7 8 | 9 2 5 | 
| 7 2 8 | 5 1 9 | 3 4 6 | 
| 5 4 9 | 3 2 6 | 8 7 1 | 
-------------------------
True
